In [1]:
using LinearAlgebra
using JuMP
using MathOptInterface
ENV["MOSEKBINDIR"] = "C:/Program Files/Mosek/8/tools/platform/win64x86/bin"
using MosekTools
using Mosek
using GLPK

## Problem 2

You are the operations manager of a dairy conglomerate that operates four dairy farms:
1. Mendota, IL
2. Havana, IL
3. Paris, IL
4. Delavan, WI

You are a contracted provider for a bottling company has five locations within the Upper Midwest:
1. Madison, WI
2. Chicago, IL
3. Indianapolis, IN
4. Saint Louis, MO
5. Detroit, MI

Your production costs for the milk average \\$21.91/cwt, and the five bottling locations are currently paying the following per-gallon wholesale price:
1. \\$2.19/gal
2. \\$2.53/gal
3. \\$2.35/gal
4. \\$2.40/gal
5. \\$2.60/gal

(Note: 1 cwt = 100lb, 1 gal of milk = 8.6lb)

Each week, you are contracted to supply *at least* 10 tons and up to 25 tons of milk to a given bottling company location. Your contracted trucking company can carry up to 5 tons of milk per truck and charges the following per-mile rates:
* 0 to 150 mi: \$1.09/mi
* 151+ mi: \$0.95/mi


Distances from Farms 1-4 to Markets 1-5 are as follows:

$\begin{bmatrix}    
    125 & 105 & 251 & 241 & 362\\
    262 & 198 & 227 & 147 & 451\\
    314 & 199 & 104 & 166 & 390\\
    59 & 98  & 285 & 352 & 387\end{bmatrix}$

This week, farms 1 through 4 have produced 6500, 3950, 5350, and 9770 gallons.  Find the optimal amounts of milk to ship from each farm to each location so as to maximize profit.

## Solution

### Unit  Conversions and Model Preparation

To simplify the final optimization function, I will convert all units into tons.  Since our goal will be to maximize a function, all production costs will be represented as negative numbers.  First, convert the cwt production costs into tonnage production costs.

$$tonCost = \frac{cwtCost}{20cwt/ton}$$

In [2]:
cwtProductionCost = [-2.19; -2.19; -2.19; -2.19]
production_costs = cwtProductionCost .* 20

4-element Array{Float64,1}:
 -43.8
 -43.8
 -43.8
 -43.8

Now, convert the wholesale pricing into price per ton.

$$tonPrice = galPrice * \frac{2000lb}{8.6lb/gal}$$

In [3]:
wholesaleGalPrice = [2.19 2.53 2.35 2.40 2.60]
wholesaleTonPrice = wholesaleGalPrice .* (2000/8.6)

1×5 Array{Float64,2}:
 509.302  588.372  546.512  558.14  604.651

Perform the same conversion on milk production output to get the tonnage available for shipment at each farm.

$$tonOutput = galOutput * \frac{8.6lb/gal}{2000lb/ton}$$

In [4]:
galOutput = [6500, 3950, 5350, 9770]
Output = galOutput .* (8.6/2000)

4-element Array{Float64,1}:
 27.95 
 16.985
 23.005
 42.011

With all milk units now measured in tons, we can compute the distance matrices needed to determine shipping costs. Since all shipments are from one fixed point to another fixed point, we can represent shipping distances at each rate as two constants for the high and low-rate distances rather than using constrained variables.  We will also establish the per-mile, per-truck charge as constant negative values.


$$        
lowRateMileage(Dist) = 
     \begin{cases}
       Dist - 150 &\quad\text{if } Dist - 150 \ge0\\
       0 &\quad\text{if } Dist - 150 < 0 \\
          \end{cases}
$$            
          $$highRateMileage = Dist - lowRateMileage$$
   

In [5]:
#establish fixed distances
Dist = [
    125 105 251 241 362;
    262 198 227 147 451;
    314 199 104 166 390;
    59 98  285 352 387
]

#establish per-mile costs for each distance
mileCostHigh = -1.09
mileCostLow = -0.95

#extract dimensions of Dist matrix, intialize split matrices
output_dims = size(Dist)
highRateMileage = zeros(output_dims)
lowRateMileage = zeros(output_dims)

#loop over Dist to built lowRate Matrix
for (index, value) in enumerate(Dist)
    if value - 150 > 0
        lowRateMileage[index] = value - 150
    else
        lowRateMileage[index] = 0
    end
end

lowRateMileage

4×5 Array{Float64,2}:
   0.0   0.0  101.0   91.0  212.0
 112.0  48.0   77.0    0.0  301.0
 164.0  49.0    0.0   16.0  240.0
   0.0   0.0  135.0  202.0  237.0

In [6]:
highRateMileage = Dist - lowRateMileage

4×5 Array{Float64,2}:
 125.0  105.0  150.0  150.0  150.0
 150.0  150.0  150.0  147.0  150.0
 150.0  150.0  104.0  150.0  150.0
  59.0   98.0  150.0  150.0  150.0

We now extract the dimensionality of the main arrays for future use.

In [7]:
#collect dimensionality of shipping for later use
Farms, Markets = size(Dist)

#create iterable arrays that represent the dimensionality of shipping matrix
#I = the number of farms
I = collect(1:Farms)
#J = the number of bottling facilities
J = collect(1:Markets)

5-element Array{Int64,1}:
 1
 2
 3
 4
 5

Since the earnings computations will require a matrix of the same dimension as the shipping matrix, we build this needed matrix by repeating the per-ton-earnings potential for a shipment to a given bottling facility across four rows.

In [8]:
Earnings = zeros(size(Dist))
for num in I
    Earnings[num,:] = wholesaleTonPrice
end
Earnings

4×5 Array{Float64,2}:
 509.302  588.372  546.512  558.14  604.651
 509.302  588.372  546.512  558.14  604.651
 509.302  588.372  546.512  558.14  604.651
 509.302  588.372  546.512  558.14  604.651

### Establishing an Optimization Function



In [9]:
#initialize model
basic_model = Model(with_optimizer(Mosek.Optimizer, QUIET=false, INTPNT_CO_TOL_DFEAS=1e-7))
#basic_model = Model(with_optimizer(GLPK.Optimizer))

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Mosek

With the model space now prepared, we can begin to construct an optimization function.  Since the number of trucks required to move the product from a given farm to a given market is logically constrained to $numTrucks \in \mathbb{Z}_{\geq 0}$, the question will be a mixed-integer problem.  We begin by establishing the fixed values needed for our constraints.  Since $x_{i,j}$ will represent the tonnage of milk shipped from Farm $i$ to bottling facility $j$ and $m_j$ will represent the demand filled at a given bottling facility, $$m_j = \sum_{i=1}^4 x_{i,j} \text{ for } j = (1,2,3,4,5)\\ s.t. 10 \leq \sum_{i=1}^4 x_i \leq 25$$

In [10]:
#establish port quantities and market demands
Demand = [25, 25, 25, 25, 25]
Floor = [10, 10, 10, 10, 10]

#M will represent units demanded in each market
@variable(basic_model, m[i=1:Markets], lower_bound = Floor[i], upper_bound = Demand[i])

5-element Array{VariableRef,1}:
 m[1]
 m[2]
 m[3]
 m[4]
 m[5]

We next prepare the $p_i$ variables. The lower bound is logically 0, and the upper bound of a given $p_i$ is equal to the milk output of Farm $i$.
$$\text{Total Output} = \sum_{i=1}^4 p_i\\ \text{s.t. } p_i \in \mathbb{R}^{+}\\ p_i \leq \text{Milk produced at Farm }i$$

In [11]:
#P will represent units available in each port
@variable(basic_model, p[i=1:Farms], lower_bound = 0, upper_bound = Output[i])

4-element Array{VariableRef,1}:
 p[1]
 p[2]
 p[3]
 p[4]

As noted above, variable $x_{i,j}$ will represent the tons of milk shipped from Farm $i$ to bottling facility $j$.  This variable is bounded by $x_{i,j} \in \mathbb{R}^{+}$ and constrained by
$$\sum^5_{j=1} x_{i,j} = p_i$$ .

In [12]:
#x[i,j] = units moved from Market[i] to Port[j] 
@variable(basic_model, x[1:Farms,1:Markets] >= 0)

#supply constraints - row-wise sum of matrix x should equal supply for a given P
for farm in I
    @constraint(basic_model, sum(x[farm,:]) == p[farm]) 
end

Given that the demand across the five bottling plants, $\sum_{j=1}^5 m_j$, is greater than the maximum farm output, $\sum_{i=1}^4 p_i$, we create a slack variable to balance the problem by capturing the excess demand.  We initialize the variable by establishing a lower bound $slack_m \in \mathbb{R}^{+}$, and then constrain the values for the slack demand by establishing that 
$$\sum_{j=1}^5 m_j - \sum_{j=1}^5 slack_j= \sum_{i=1}^4 p_i$$

In [13]:
#create slack variable array for excess demand
@variable(basic_model, slackdemand[i=1:Markets], lower_bound = 0)

#demand constraints - column-wise sum of matrix x should equal market demand for a given M
for market in J
    @constraint(basic_model, sum(x[:,market]) + slackdemand[market] == Demand[market])
end

Lastly, we create variable $numTrucks_{i,j}$ that will represent the number of trucks moving milk from Farm $i$ to bottling facility $j$.  Since the cost function for shipping is per-truck and not per-unit, we bound the number of trucks to integer values: 
$$numTrucks \in \mathbb{Z}_{\geq 0}$$

Additionally, since each truck can carry up to five tons of milk each, we constrain $numTrucks_{i,j}$ relative to the amount of milk moving from Farm $i$ to bottling facility $j$ thusly:

$$numTrucks_{i,j} \geq \frac{1}{5} * x_{i,j}$$

In [14]:
#variable space for number of trucks required, Int constraint
@variable(basic_model,numTrucks[1:Farms,1:Markets], lower_bound = 0, Int)

#number of trucks constraint
for (index, value) in enumerate(Dist)
    @constraint(basic_model, numTrucks[index] >= (1/5) * x[index])
end

With the variables established and fully bounded, we can now build the objective function.  

$$ \text{max} \sum_{i=1}^4 p_i \cdot \text{production_cost} + \sum_{i,j = 0}^{4,5} -1.09 \cdot \text{highRateMileage}_{i,j} \cdot numTrucks_{i,j} +\\ \sum_{i,j = 0}^{4,5} -0.95 \cdot \text{lowRateMileage}_{i,j} \cdot numTrucks_{i,j} + \sum_{i,j = 0}^{4,5} \text{Earnings}_{i,j} \cdot x_{i,j} $$ 

In [15]:
#compose the objective function
@objective(basic_model, Max, sum(production_costs .* p) + sum(highRateMileage.*mileCostHigh.*numTrucks) 
    + sum(lowRateMileage.*mileCostLow.*numTrucks) + sum(Earnings .* x))

-43.8 p[1] - 43.8 p[2] - 43.8 p[3] - 43.8 p[4] - 136.25 numTrucks[1,1] - 269.9 numTrucks[2,1] - 319.29999999999995 numTrucks[3,1] - 64.31 numTrucks[4,1] - 114.45 numTrucks[1,2] - 209.1 numTrucks[2,2] - 210.05 numTrucks[3,2] - 106.82000000000001 numTrucks[4,2] - 259.45 numTrucks[1,3] - 236.64999999999998 numTrucks[2,3] - 113.36000000000001 numTrucks[3,3] - 291.75 numTrucks[4,3] - 249.95 numTrucks[1,4] - 160.23000000000002 numTrucks[2,4] - 178.7 numTrucks[3,4] - 355.4 numTrucks[4,4] - 364.9 numTrucks[1,5] - 449.45 numTrucks[2,5] - 391.5 numTrucks[3,5] - 388.65 numTrucks[4,5] + 509.3023255813954 x[1,1] + 509.3023255813954 x[2,1] + 509.3023255813954 x[3,1] + 509.3023255813954 x[4,1] + 588.3720930232558 x[1,2] + 588.3720930232558 x[2,2] + 588.3720930232558 x[3,2] + 588.3720930232558 x[4,2] + 546.5116279069769 x[1,3] + 546.5116279069769 x[2,3] + 546.5116279069769 x[3,3] + 546.5116279069769 x[4,3] + 558.1395348837209 x[1,4] + 558.1395348837209 x[2,4] + 558.1395348837209 x[3,4] + 558.139534883

In [16]:
@time begin
    status = optimize!(basic_model)
end

Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : LO (linear optimization problem)
  Constraints            : 29              
  Cones                  : 0               
  Scalar variables       : 54              
  Matrix variables       : 0               
  Integer variables      : 20              

Optimizer started.
Mixed integer optimizer started.
Threads used: 4
Presolve started.
Presolve terminated. Time = 0.00
Presolved problem: 40 variables, 29 constraints, 80 non-zeros
Presolved problem: 20 general integer, 0 binary, 20 continuous
Clique table size: 0
BRANCHES RELAXS   ACT_NDS  DEPTH    BEST_INT_OBJ         BEST_RELAX_OBJ       REL_GAP(%)  TIME  
0        1        0        0        NA                   5.3616992827e+04     NA          0.1   
0        1        0        0        5.2948291549e+04     5.3616992827e+04     1.26        0.1   
0        1        0        0        5.3300486223e+04     5.36169928

In [17]:
#print objective value
println("Earnings: \$", JuMP.objective_value(basic_model))

Earnings: $53333.76713023257


In [18]:
#print tons moved and trucks used
display("Tons moved:")
display(JuMP.value.(x))
display("Trucks required:")
display(JuMP.value.(numTrucks))

"Tons moved:"

4×5 Array{Float64,2}:
  0.0     0.0   0.0           7.95         20.0        
  0.0     0.0   6.66134e-15  16.985         1.03989e-14
  0.0     0.0  23.005        -3.55271e-15   0.0        
 12.011  25.0   0.0           0.0           5.0        

"Trucks required:"

4×5 Array{Float64,2}:
 0.0  0.0  0.0   2.0          4.0
 0.0  0.0  0.0   4.0          0.0
 0.0  0.0  5.0  -7.10543e-16  0.0
 3.0  5.0  0.0   0.0          1.0

In [19]:
#dump all vars to console
list = JuMP.all_variables(basic_model)

#now iterate over list and print values
for name in list
    println(name,":", JuMP.value(name))
end

m[1]:10.0
m[2]:10.0
m[3]:10.0
m[4]:10.0
m[5]:10.0
p[1]:27.949999999999996
p[2]:16.985
p[3]:23.005
p[4]:42.011
x[1,1]:0.0
x[1,2]:0.0
x[1,3]:0.0
x[1,4]:7.950000000000036
x[1,5]:19.99999999999996
x[2,1]:0.0
x[2,2]:0.0
x[2,3]:6.661338147750935e-15
x[2,4]:16.98499999999998
x[2,5]:1.0398904880734458e-14
x[3,1]:0.0
x[3,2]:0.0
x[3,3]:23.005000000000003
x[3,4]:-3.552713678800501e-15
x[3,5]:0.0
x[4,1]:12.010999999999974
x[4,2]:25.0
x[4,3]:0.0
x[4,4]:0.0
x[4,5]:5.000000000000027
slackdemand[1]:12.989000000000026
slackdemand[2]:0.0
slackdemand[3]:1.9949999999999903
slackdemand[4]:0.06499999999998707
slackdemand[5]:0.0
numTrucks[1,1]:0.0
numTrucks[1,2]:0.0
numTrucks[1,3]:0.0
numTrucks[1,4]:2.0
numTrucks[1,5]:3.9999999999999925
numTrucks[2,1]:0.0
numTrucks[2,2]:0.0
numTrucks[2,3]:0.0
numTrucks[2,4]:3.9999999999999907
numTrucks[2,5]:0.0
numTrucks[3,1]:0.0
numTrucks[3,2]:0.0
numTrucks[3,3]:5.0
numTrucks[3,4]:-7.105427357601002e-16
numTrucks[3,5]:0.0
numTrucks[4,1]:3.0
numTrucks[4,2]:5.0
numTrucks[4,3]

In [20]:
#use two variable sets for shipping distances travelled
#@variable(basic_model,truckDistHigh[1:Farms,1:Markets], lower_bound = 0, upper_bound = 150)
#@variable(basic_model,truckDistLow[1:Farms,1:Markets], lower_bound = 0)

#shipping cost constraints
#low cost is value - 150, noting that dist is constrainted to be >= 0 in declaration
#for (index, value) in enumerate(Dist)
#    @constraint(basic_model, truckDistLow[index] >= value - 150)
#end

#high shipping costs contraints
#for (index, value) in enumerate(Dist)
#    @constraint(basic_model, truckDistHigh[index] == value - truckDistLow[index])
#end

### Question 2b

The owners of the farm collective are curious if they can make more money under current market conditions by reallocating their milk cows to different farms so as to reduce overall shipping costs.  The current cattle distribution is as follows:

1. 142 head
2. 87 head
3. 118 head
4. 215 head

Each farm needs to have at least 75 head of cattle, and no farm can hold more than 250 head. Each cow produces on average 45.5gal milk per week.

We start by reestablishing the previous model deployed:

In [21]:
basic_model = Model(with_optimizer(Mosek.Optimizer, QUIET=false, INTPNT_CO_TOL_DFEAS=1e-7))

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Mosek

We add in a new variable, $numCows \in \mathbb{Z}_{\geq 0}$, such that $75 \leq numCows_i \leq 250 \text{ for } i = (1,2,3,4)$ and that
$$\sum^4_{i=1} numCows_i \leq 562$$

In [22]:
@variable(basic_model, numCows[i=1:Farms], lower_bound = 75, upper_bound = 250, Int)
@constraint(basic_model, sum(numCows) <= 562)

numCows[1] + numCows[2] + numCows[3] + numCows[4] <= 562.0

We then modify the previous output variable, $p$, to derive the tons of milk produced by $numCows_i$ at Farm $i$:

$$p_i = 45.5 * numCows_i * \frac{8.6lb/gal}{2000lb}$$

In [23]:
@variable(basic_model, p[i=1:Farms], lower_bound = 0)
for farm in I
    @constraint(basic_model, p[farm] == (numCows[farm] * 45.5) * (8.6/2000)) 
end

The optimization question is otherwise as before:

In [24]:
@variable(basic_model, m[i=1:Markets], lower_bound = Floor[i], upper_bound = Demand[i])
@variable(basic_model, x[1:Farms,1:Markets] >= 0)

for farm in I
    @constraint(basic_model, sum(x[farm,:]) == p[farm]) 
end

@variable(basic_model, slackdemand[i=1:Markets], lower_bound = 0)

for market in J
    @constraint(basic_model, sum(x[:,market]) + slackdemand[market] == Demand[market])
end

@variable(basic_model,numTrucks[1:Farms,1:Markets], lower_bound = 0, Int)

for (index, value) in enumerate(Dist)
    @constraint(basic_model, numTrucks[index] >= (1/5) * x[index])
end

In [25]:
@objective(basic_model, Max, sum(production_costs .* p) + sum(highRateMileage.*mileCostHigh.*numTrucks) 
    + sum(lowRateMileage.*mileCostLow.*numTrucks) + sum(Earnings .* x))

-43.8 p[1] - 43.8 p[2] - 43.8 p[3] - 43.8 p[4] - 136.25 numTrucks[1,1] - 269.9 numTrucks[2,1] - 319.29999999999995 numTrucks[3,1] - 64.31 numTrucks[4,1] - 114.45 numTrucks[1,2] - 209.1 numTrucks[2,2] - 210.05 numTrucks[3,2] - 106.82000000000001 numTrucks[4,2] - 259.45 numTrucks[1,3] - 236.64999999999998 numTrucks[2,3] - 113.36000000000001 numTrucks[3,3] - 291.75 numTrucks[4,3] - 249.95 numTrucks[1,4] - 160.23000000000002 numTrucks[2,4] - 178.7 numTrucks[3,4] - 355.4 numTrucks[4,4] - 364.9 numTrucks[1,5] - 449.45 numTrucks[2,5] - 391.5 numTrucks[3,5] - 388.65 numTrucks[4,5] + 509.3023255813954 x[1,1] + 509.3023255813954 x[2,1] + 509.3023255813954 x[3,1] + 509.3023255813954 x[4,1] + 588.3720930232558 x[1,2] + 588.3720930232558 x[2,2] + 588.3720930232558 x[3,2] + 588.3720930232558 x[4,2] + 546.5116279069769 x[1,3] + 546.5116279069769 x[2,3] + 546.5116279069769 x[3,3] + 546.5116279069769 x[4,3] + 558.1395348837209 x[1,4] + 558.1395348837209 x[2,4] + 558.1395348837209 x[3,4] + 558.139534883

In [26]:
@time begin
    status = optimize!(basic_model)
end

Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : LO (linear optimization problem)
  Constraints            : 34              
  Cones                  : 0               
  Scalar variables       : 58              
  Matrix variables       : 0               
  Integer variables      : 24              

Optimizer started.
Mixed integer optimizer started.
Threads used: 4
Presolve started.
Presolve terminated. Time = 0.00
Presolved problem: 44 variables, 30 constraints, 88 non-zeros
Presolved problem: 24 general integer, 0 binary, 20 continuous
Clique table size: 0
BRANCHES RELAXS   ACT_NDS  DEPTH    BEST_INT_OBJ         BEST_RELAX_OBJ       REL_GAP(%)  TIME  
0        1        0        0        NA                   5.3841480698e+04     NA          0.0   
0        1        0        0        5.3285505767e+04     5.3841480698e+04     1.04        0.0   
0        1        0        0        5.3425573209e+04     5.38414806

In [27]:
#print objective value
println("Earnings: \$", JuMP.objective_value(basic_model))

Earnings: $53750.66495302327


In [28]:
#print tons moved and trucks used
display("Tons moooooved:")
display(JuMP.value.(x))
display("Trucks required:")
display(JuMP.value.(numTrucks))

"Tons moved:"

4×5 Array{Float64,2}:
  0.0      4.93445   0.0      0.0     25.0
  0.0      0.0       0.0     24.8475   0.0
  0.0      0.0      24.8476   0.0      0.0
 10.3257  20.0       0.0      0.0      0.0

"Trucks required:"

4×5 Array{Float64,2}:
 0.0  1.0  0.0  0.0  5.0
 0.0  0.0  0.0  5.0  0.0
 0.0  0.0  5.0  0.0  0.0
 3.0  4.0  0.0  0.0  0.0

In [29]:
#dump all vars to console
list = JuMP.all_variables(basic_model)

#now iterate over list and print values
for name in list
    println(name,":", JuMP.value(name))
end

numCows[1]:153.0
numCows[2]:127.0
numCows[3]:127.00000000000001
numCows[4]:155.0
p[1]:29.93445
p[2]:24.84755
p[3]:24.847550000000002
p[4]:30.32575
m[1]:10.0
m[2]:10.0
m[3]:10.0
m[4]:10.0
m[5]:10.0
x[1,1]:0.0
x[1,2]:4.934449999999998
x[1,3]:0.0
x[1,4]:0.0
x[1,5]:25.0
x[2,1]:0.0
x[2,2]:0.0
x[2,3]:0.0
x[2,4]:24.84755
x[2,5]:0.0
x[3,1]:0.0
x[3,2]:0.0
x[3,3]:24.847550000000002
x[3,4]:0.0
x[3,5]:0.0
x[4,1]:10.32575
x[4,2]:20.0
x[4,3]:0.0
x[4,4]:0.0
x[4,5]:0.0
slackdemand[1]:14.67425
slackdemand[2]:0.06555000000000177
slackdemand[3]:0.1524499999999982
slackdemand[4]:0.15245000000000175
slackdemand[5]:0.0
numTrucks[1,1]:0.0
numTrucks[1,2]:1.0
numTrucks[1,3]:0.0
numTrucks[1,4]:0.0
numTrucks[1,5]:5.0
numTrucks[2,1]:0.0
numTrucks[2,2]:0.0
numTrucks[2,3]:0.0
numTrucks[2,4]:5.0
numTrucks[2,5]:0.0
numTrucks[3,1]:0.0
numTrucks[3,2]:0.0
numTrucks[3,3]:5.0
numTrucks[3,4]:0.0
numTrucks[3,5]:0.0
numTrucks[4,1]:3.0
numTrucks[4,2]:4.0
numTrucks[4,3]:0.0
numTrucks[4,4]:0.0
numTrucks[4,5]:0.0
